# DQN On Foreign Exchange Market
## Load dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import time
import copy
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
from collections import deque

init_notebook_mode()

Flags for the debuging purposes are presented here.

In [ ]:
# If this flag is set, each step of the environment's state will be printed
ENVIRONMENT_DEBUG = False

In [ ]:
ds = pd.read_csv(
    "https://static-1300131294.cos.ap-shanghai.myqcloud.com/data/deep-learning/dqn/USD_TRY%20Gemi%20Verileri.csv"
)
ds["Tarih"] = pd.to_datetime(ds["Tarih"], errors="coerce")
ds["Şimdi"] = pd.to_numeric(ds["Şimdi"].str.replace(",", "."), errors="coerce")
ds["Fark %"] = ds["Fark %"].str.replace("%", "")
ds["Fark %"] = ds["Fark %"].str.replace(",", ".")

ds.head(10)

The reverse version of the data will be used. It is expected to make the learning stronger since from 2002 to this date, usd is increasing with respect to try.

In [ ]:
X = ds["Şimdi"]
Y = ds["Tarih"]
X = np.array(X).reshape((len(X), 1))
Y = np.array(Y).reshape((len(Y), 1))
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot_date(Y, X, ".")
plt.show()

In [ ]:
# Split train and test data

date_split = 4000
train = ds[:date_split]
test = ds[date_split:]
len(train), len(test)

## Define envireonment

In [ ]:
class Environment:
    def __init__(self, data, tl, history_t=10):
        self.data = data
        self.history_t = history_t
        self.tl_start = tl
        self.reset()

    def reset(self):
        self.tl = self.tl_start
        self.usd = 0
        self.done = False
        self.profits = 0
        self.current_position = "none"
        self.history = [self.data.iloc[x, :]["Fark %"] for x in range(self.history_t)]
        self.t = self.history_t
        self.last_tl = 0
        return self.history

    def step(self, act):
        reward = 0
        if act == 0:  # Hold
            self.current_position = self.current_position
        elif act == 1:  # Buy
            if self.current_position == "none":
                self.current_position = "long"
                # Buy usd

                self.last_tl = self.tl
                self.usd = self.tl / (self.data.iloc[self.t, :]["Şimdi"])
                self.tl = 0
            else:
                self.current_position = self.current_position

        else:  # sell
            if self.current_position == "long":
                self.current_position = "none"
                # Sell usd

                self.tl = self.usd * (self.data.iloc[self.t, :]["Şimdi"])
                self.usd = 0
                self.profits = self.profits + (self.tl - self.last_tl)

                if (self.tl - self.last_tl) > 0:
                    reward = 1
                else:
                    reward = -1
            else:
                self.current_position = self.current_position

        # et next time
        self.t += 1
        # print("history before: ",self.history)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]["Fark %"])
        # print("history after: ",self.history)

        # print("reward: ",reward)

        if ENVIRONMENT_DEBUG == True:
            print(
                "t: ",
                (self.t - self.history_t),
                " reward: ",
                reward,
                " profits: ",
                self.profits,
                " current position: ",
                self.current_position,
                " done: ",
                self.done,
            )

        if self.t == (len(self.data) - 1):
            self.done = True
            print(
                "Total steps: ",
                (self.t - self.history_t),
                " TotalProfit: ",
                self.profits,
                " done: ",
                self.done,
            )

        return self.history, reward, self.done, self.profits  # obs, reward, done

## Agent class

In [ ]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size, hidden_layer_size):
        self.state_size = state_size
        self.action_size = action_size
        self.hidden_layer_size = hidden_layer_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(
            Dense(self.hidden_layer_size, input_dim=self.state_size, activation="relu")
        )
        model.add(Dense(self.hidden_layer_size, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        state = np.array(state).astype(float)
        state = np.reshape(state, [1, self.state_size])
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def act_greedy(self, state):
        state = np.array(state).astype(float)
        state = np.reshape(state, [1, self.state_size])
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(
                    self.model.predict(next_state)[0]
                )
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

## Train the DQN

In [ ]:
if __name__ == "__main__":
    # macros
    EPISODES = 50
    STATE_SIZE = 90

    # profits list
    total_profits = []

    # initialize environment and the agent
    env = Environment(train, 100, STATE_SIZE)  # 100tl, 60 history
    agent = DQNAgent(STATE_SIZE, 3, 100)

    # Iterate the game
    for e in range(EPISODES):
        # check if the buy and sell actions are taken
        actions_count = 0

        # reset state in the beginning of each game
        state = env.reset()
        state = np.reshape(state, [1, STATE_SIZE])

        # time_t represents each frame of the game
        # the more time_t the more score
        for time_t in range(5000):
            # Decide action
            action = agent.act(state)

            if (action == 1) or (action == 2):
                actions_count = actions_count + 1

            # Advance the game to the next frame based on the action.
            next_state, reward, done, profits = env.step(action)
            next_state = np.array(next_state, dtype=np.float32)
            next_state = np.reshape(next_state, [1, STATE_SIZE])

            # make rewards = profits (EXPERIMENTAL)
            reward = profits

            # Remember the previous state, action, reward, and done
            agent.remember(state, action, reward, next_state, done)

            # make next_state the new current state for the next frame.
            state = next_state

            # done becomes True when the game ends
            if done:
                total_profits.append(profits)
                # print the score and break out of the loop
                # print("number of actions taken other than hold in this iteration is ",actions_count,"\n")
                # print("episode: {}/{}, score: {}".format(e, EPISODES, time_t))
                break
        # train the agent with the experience of the episode
        agent.replay(32)

In [ ]:
def plot_profits(total_profits):
    epoch_count = range(1, len(total_profits) + 1)
    fig = plt.figure(figsize=(30, 10))
    plt.plot(epoch_count, total_profits, "b-")
    plt.legend("Total Profits")
    plt.xlabel("Epoch")
    plt.ylabel("Total Profits")
    plt.figure(figsize=(50, 10))
    plt.show();

In [ ]:
plot_profits(total_profits)

Now test the agent with real world data.

In [ ]:
test = test.iloc[::-1]
test.head(10)

In [ ]:
env_test = Environment(test, 100, STATE_SIZE)  # 100tl, 60 history

# Iterate the game
for e in range(1):
    # check if the buy and sell actions are taken
    actions_count = 0

    # reset state in the beginning of each game
    state = env_test.reset()
    state = np.reshape(state, [1, STATE_SIZE])

    # time_t represents each frame of the game
    # the more time_t the more score
    for time_t in range(5000):
        # Decide action
        action = agent.act_greedy(state)

        if (action == 1) or (action == 2):
            actions_count = actions_count + 1

        # Advance the game to the next frame based on the action.
        next_state, reward, done, profits = env_test.step(action)

        next_state = np.reshape(next_state, [1, STATE_SIZE])

        # make rewards = profits (EXPERIMENTAL)
        reward = profits

        # Remember the previous state, action, reward, and done
        agent.remember(state, action, reward, next_state, done)

        # make next_state the new current state for the next frame.
        state = next_state

        # done becomes True when the game ends
        if done:
            break

## Acknowledgement
Thanks to [emrebulbul23](https://www.kaggle.com/emrebulbul23) for creating [DQN on foreign exchange market](https://www.kaggle.com/code/emrebulbul23/dqn-on-foreign-exchange-market). It inspired the majority of the content in this article.